In [2]:
from time import time
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import svm

from grakel import datasets, GraphKernel, graph_from_networkx

### Load datasets

In [3]:
dataset_name = "easy_small" # options: {easy_small, easy, hard_small, hard}

loaded = np.load('datasets/'+dataset_name+'.npz', allow_pickle=True)
G_tr = [_ for _ in loaded['tr_graphs']] # networkx graphs
y_train = loaded['tr_class'] 
G_te = [_ for _ in loaded['te_graphs']] # networkx graphs
y_test = loaded['te_class']

# Put the data in GraKel format
G_train = graph_from_networkx(G_tr, node_labels_tag='features')
G_train = [g for g in G_train]
y_train = np.argmax(y_train, axis=-1)
G_test = graph_from_networkx(G_te, node_labels_tag='features')
G_test = [g for g in G_test]
y_test = np.argmax(y_test, axis=-1)

### Train and evaluate graph kernels

In [4]:
# Create a list with the graph kernels to evaluate
# For more kernels and information:
# https://ysig.github.io/GraKeL/dev/generated/grakel.GraphKernel.html#grakel.GraphKernel
kernel_names = [
                "shortest_path", 
                "graphlet_sampling", 
                "pyramid_match", 
                 "svm_theta",
                "neighborhood_hash",
                "subtree_wl",
                "odd_sth",
                "propagation",
                "pyramid_match",
                "vertex_histogram",
                "weisfeiler_lehman",
                "hadamard_code",
                "core_framework"
                ]

for k_ in kernel_names:
    
    start = time()
    
    # General kernels
    if k_ in ["weisfeiler_lehman" , "core_framework"]:
        gk = GraphKernel(kernel=[{"name": k_}, {"name": "shortest_path"}], normalize=True)
    
    # Base kernels
    else:
        gk = GraphKernel(kernel=[{"name": k_}], normalize=True)

    # Calculate the kernel matrix
    K_train = gk.fit_transform(G_train)
    K_test = gk.transform(G_test)
    
    # Initialise an SVM and fit
    clf = svm.SVC(kernel='precomputed', C=1)
    clf.fit(K_train, y_train)
    
    # Compute predictions on test set
    y_pred = clf.predict(K_test)
    
    # Calculate accuracy of classification
    acc = accuracy_score(y_test, y_pred)
    
    end = time()
    print(k_, "-- Accuracy:", str(round(acc*100, 2)), "% | Took:",
          str(round(end - start, 2)), "s")

shortest_path -- Accuracy: 97.37 % | Took: 20.15 s
graphlet_sampling -- Accuracy: 42.11 % | Took: 295.23 s
pyramid_match -- Accuracy: 44.74 % | Took: 3.71 s
svm_theta -- Accuracy: 39.47 % | Took: 4.01 s
neighborhood_hash -- Accuracy: 97.37 % | Took: 3.99 s
subtree_wl -- Accuracy: 39.47 % | Took: 0.02 s
odd_sth -- Accuracy: 65.79 % | Took: 61.98 s
propagation -- Accuracy: 84.21 % | Took: 3.18 s
pyramid_match -- Accuracy: 44.74 % | Took: 2.95 s
vertex_histogram -- Accuracy: 39.47 % | Took: 0.01 s
weisfeiler_lehman -- Accuracy: 97.37 % | Took: 141.3 s
core_framework -- Accuracy: 97.37 % | Took: 59.7 s
